In [129]:
import pandas as pd
import plotly.express as px
import numpy as np
import streamlit as st
import matplotlib.cm as cm

dg_key = "e297e933c3ad47d71ec1626c299e"
market_type = 'top_5'

names_dict = {'Matt Fitzpatrick': 'Matthew Fitzpatrick',
    'Si Kim': 'Si Woo Kim',
    'Min Lee': 'Min Woo Lee',
    'Byeong An': 'Byeong Hun An',
    'Rooyen Van': 'Erik Van Rooyen',
    'Vince Whaley': 'Vincent Whaley',
    'Kevin Yu': 'Kevin Yu',
    'Kyounghoon Lee': 'Kyoung-Hoon Lee'
             }

def implied_probability(moneyline_odds):
    """
    Calculate the implied probability from moneyline odds.
    """
    # Convert negative odds to positive for calculation
    if moneyline_odds < 0:
        implied_prob = abs(moneyline_odds) / (abs(moneyline_odds) + 100)
    else:
        implied_prob = 100 / (moneyline_odds + 100)
    
    # Return implied probability as percentage
    return implied_prob

def fix_names(list_of_player_names):
    
    # swaps last/first into first/last name
    names = list_of_player_names.str.split(expand=True)                  
    names[0] = names[0].str.rstrip(",")
    names[1] = names[1].str.rstrip(",")
    names['player_name'] = names[1].str[0] + " " + names[0]

    # uses dictionary to correct known problem names
    # ie "Jr" or "Si Woo Kim")
    for incorrect_name, correct_name in names_dict.items():
        names['player_name'] = np.where(names['player_name'] == incorrect_name, correct_name, names['player_name'])

    return names.player_name

def plus_prefix(a):
    if a > 0:
        return f"+{a}"
    return a

market_target_dict = {
    'win':1.15,
    'top_5': 1.1,
    'top_10': 1.1,
    'top_20': 1.05
    }


def convert_euro_to_american(dec_odds):
    if dec_odds >= 2:
        return (dec_odds - 1) * 100
    else:
        return -100 / (dec_odds-1)
    
#     Function to generate CSS styles for background gradient based on column values
def background_gradient(val):
    # Assuming you want to map the gradient from blue to red based on the Color_Column values
    # You can adjust the colors and the gradient range as needed
    color = f'rgba(0, 0, {(val / df["ev"].max()) * 255}, 1)' 
    return f'background-color: {color}'

In [152]:

# api calls
dg_american = pd.read_csv(f"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market={market_type}&odds_format=american&file_format=csv&key={dg_key}")
dg_decimal = pd.read_csv(f"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market={market_type}&odds_format=decimal&file_format=csv&key={dg_key}")

# grab american and euro DataGolf odds for each player and combine
am_odds = dg_american[['player_name','datagolf_base_history_fit']].rename(columns={'datagolf_base_history_fit':'am_odds_dg'})
dec_odds = dg_decimal[['player_name','datagolf_base_history_fit']].rename(columns={'datagolf_base_history_fit':'dec_odds_dg'})
dg_odds = pd.merge(am_odds,dec_odds, on='player_name')

# get best sportbooks line for each player in american and euro formats
agg_am = dg_american.drop(columns=['player_name','dg_id','datagolf_baseline','datagolf_base_history_fit','last_updated','event_name','market']).T.max().to_frame()
agg_dec = dg_decimal.drop(columns=['player_name','dg_id','datagolf_baseline','datagolf_base_history_fit','last_updated','event_name','market']).T.max().to_frame()

# agg_am = dg_american[books].T.max().to_frame()
# agg_dec = dg_decimal[books].T.max().to_frame()

# combine and fix column names
df = pd.merge(dg_odds, agg_am, left_index=True, right_index=True)
df = pd.merge(df,agg_dec,left_index=True, right_index=True).T.drop_duplicates().T
df.columns = ['player_name', 'dg_american','dg_decimal','books_mean_american','books_mean_decimal']

# convert target euro odds to american for display
df['market_type'] = market_type
df['market_target'] = df['market_type'].map(market_target_dict)
df['target_decimal'] = (df['dg_decimal'] * df['market_target']).astype(float)
df['target_american'] = df['target_decimal'].apply(convert_euro_to_american).astype(int)

# add expected value column (for color)
df['ev'] = ((1 / df['dg_decimal']) * df['books_mean_decimal'] -1).astype(float)

# # flip first/last player names
# df['player_name'] = fix_names(df['player_name'])

# # column names
# df = df[['player_name','am_odds','ev','target_american']]
# df = df.convert_dtypes()#.round(2)
df[:5]
# Apply the background_gradient function to the Value_Column
# styled_df = df.style.applymap(background_gradient, subset=['am_odds'])

# styled_df#.info()



,player_name,dg_american,dg_decimal,books_mean_american,books_mean_decimal,market_type,market_target,target_decimal,target_american,ev
0,"McIlroy, Rory",247,3.472176,200,3.0,top_5,1.1,3.819393,281,-0.135988
1,"Schauffele, Xander",286,3.862522,250,3.5,top_5,1.1,4.248774,324,-0.093856
2,"Clark, Wyndham",473,5.731513,450,5.5,top_5,1.1,6.304664,530,-0.040393
3,"Morikawa, Collin",592,6.920372,550,6.5,top_5,1.1,7.612409,661,-0.060744
4,"Cantlay, Patrick",585,6.852893,450,5.5,top_5,1.1,7.538182,653,-0.197419


In [ ]:
def get_ev_table_():
    

In [135]:
temp = (
    df
    [['player_name','target_american','target_decimal','ev']]
    .astype({'target_decimal':'int'})
    .round({'target_american':-1})
    .assign(player_name=lambda x: fix_names(x['player_name']))
)

# # column names
american = temp[['player_name','target_american','ev']]
american.columns = ['Player','Target','EV']
# american = american.convert_dtypes()

decimal = temp[['player_name','target_decimal','ev']]
decimal.columns = ['Player','Target','EV']
# decimal = decimal.convert_dtypes()

# Apply the background_gradient function to the Value_Column
styled_american = american.style.background_gradient(subset=['Target']).format(precision=2)
styled_decimal = decimal.style.background_gradient(subset=['Target']).format(precision=2)


In [136]:
styled_american

,Player,Target,EV
0,R McIlroy,280,-0.14
1,X Schauffele,320,-0.09
2,W Clark,530,-0.13
3,C Morikawa,660,-0.06
4,P Cantlay,650,-0.20
5,J Thomas,690,-0.03
6,S Theegala,710,-0.05
7,T Fleetwood,780,-0.12
8,V Hovland,830,-0.17
9,T Finau,840,-0.07


In [134]:
styled_decimal

,Player,Target,EV
0,R McIlroy,3,-0.14
1,X Schauffele,4,-0.09
2,W Clark,6,-0.13
3,C Morikawa,7,-0.06
4,P Cantlay,7,-0.20
5,J Thomas,7,-0.03
6,S Theegala,8,-0.05
7,T Fleetwood,8,-0.12
8,V Hovland,9,-0.17
9,T Finau,9,-0.07


In [4]:


# Function to generate CSS styles for background gradient based on column values
def background_gradient(val):
    # Map the value to the range [0, 1]
    normalized_val = (val - df["ev"].min()) / (df["ev"].max() - df["ev"].min())

    colors = [
        (0.000000, 0.126245, 0.301561),   # Dark blue
        (0.008908, 0.141791, 0.332153),
        (0.022032, 0.154709, 0.359833),
        (0.038399, 0.165691, 0.385092),
        (0.056929, 0.175116, 0.408207),
        (0.075353, 0.183631, 0.429814),
        (0.092500, 0.191676, 0.450855),
        (0.108038, 0.199638, 0.471214),
        (0.121928, 0.207695, 0.490858),
        (0.134894, 0.216049, 0.509770),
        (0.147787, 0.225117, 0.527964),
        (0.161339, 0.235392, 0.545462),
        (0.176070, 0.247441, 0.562293),
        (0.192441, 0.261847, 0.578488),
        (0.210730, 0.278257, 0.594091),
        (0.231078, 0.296292, 0.609142),
        (0.253988, 0.315650, 0.623689),
        (0.279045, 0.336015, 0.637779),
        (0.305595, 0.357137, 0.651457),
        (0.332932, 0.378798, 0.664771),
        (0.360889, 0.401756, 0.677761),
        (0.389758, 0.424936, 0.690466),
        (0.420094, 0.448168, 0.702926),
        (0.452574, 0.471785, 0.715178),
        (0.486903, 0.495845, 0.727256),
        (0.522080, 0.520073, 0.739194),
        (0.557862, 0.544620, 0.751022),
        (0.594818, 0.569755, 0.762770),
        (0.632451, 0.595294, 0.774471),
        (0.670253, 0.621315, 0.786150),
        (0.707666, 0.647830, 0.797830),
        (0.744098, 0.674438, 0.809513),
        (0.779950, 0.701247, 0.821218),
        (0.815469, 0.728571, 0.832972),
        (0.851001, 0.756559, 0.844807),
        (0.887173, 0.785380, 0.856760),
        (0.924157, 0.815155, 0.868807),
        (0.961888, 0.846102, 0.880792),  # Light yellow
    ]

    # Interpolate color based on normalized value
    index = int(normalized_val * (len(colors) - 1))
    color = colors[index]
    
    # Convert RGB tuple to CSS format
    color_css = f'rgba({int(color[0]*255)}, {int(color[1]*255)}, {int(color[2]*255)}, 1)'
    
    return f'background-color: {color_css}'

# Apply the background_gradient function to the Value_Column
styled_df = df.style.applymap(background_gradient, subset=['am_odds'])

# Display the styled DataFrame
styled_df


IndexError: list index out of range

In [ ]:
df.style.background_gradient(cmap="gist_heat", subset=['ev'])

In [5]:
dg_odds = df.copy()

# fix names and column headers
dg_odds['player_name'] = fix_names(dg_odds)
# dg_odds['real_odds'] = dg_odds['real_odds'].apply(plus_prefix)
# dg_odds['agg_line'] = dg_odds['agg_line'].apply(plus_prefix)

dg_odds.columns = ['Player','Odds','Agg','EV']
dg_odds.sort_values('EV',ascending=True)

# styled_df = dg_odds.style.background_gradient(cmap="gist_heat", subset=['EV']).format(precision=2)
# styled_df

AttributeError: 'DataFrame' object has no attribute 'str'

In [ ]:
dg_outrights_data = pd.read_csv(f"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market={market_type}&odds_format=american&file_format=csv&key={dg_key}")
books = ['betmgm', 'betfair', 'fanduel', 'draftkings', 'bovada','williamhill', 'betonline', 'unibet', 'bet365','betway', 'skybet', 'pointsbet']

# dg and book aggregates in moneyline form for all players all markets
real_odds = dg_outrights_data[['player_name','datagolf_base_history_fit']]
agg_lines = dg_outrights_data[books].T.mean().to_frame() 
dg_odds = pd.merge(real_odds, agg_lines, left_index=True, right_index=True) 

# dg decimal odds for calculating ev
dec_odds = pd.read_csv(f"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market={market_type}&odds_format=decimal&file_format=csv&key={dg_key}",usecols=['player_name','datagolf_base_history_fit'])

# merge
df = dg_odds.merge(dec_odds,left_index=True, right_index=True).rename(
            columns={
                # 'player_name':'player',
                'datagolf_base_history_fit':'real_odds',
                '0_x':'agg_line',
                '0_y':'agg_dec'
                }
                )
df

In [ ]:
dg_outrights_data = pd.read_csv(f"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market={market_type}&odds_format=american&file_format=csv&key={dg_key}")
books = ['betmgm', 'betfair', 'fanduel', 'draftkings', 'bovada','williamhill', 'betonline', 'unibet', 'bet365','betway', 'skybet', 'pointsbet']

# dg and book aggregates in moneyline form for all players all markets
real_odds = dg_outrights_data[['player_name','datagolf_base_history_fit']]
agg_lines = dg_outrights_data[books].T.mean().to_frame() 
dg_odds = pd.merge(real_odds, agg_lines, left_index=True, right_index=True) 

# dg decimal odds for calculating ev
dec_odds = pd.read_csv(f"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market={market_type}&odds_format=decimal&file_format=csv&key={dg_key}",usecols=['player_name','datagolf_base_history_fit'])

# merge
df = dg_odds.merge(dec_odds,left_index=True, right_index=True).T.drop_duplicates().T

df.columns = ['player_name','real_odds','agg_line','euro_line']
df = df.dropna()

df['implied_prob'] = df['real_odds'].apply(implied_probability)
df['ev'] = df['implied_prob'] * df['euro_line'] -1
df = df[['player_name','real_odds','agg_line','ev']].convert_dtypes().round(2)

df